# Preprocessamento

In [30]:
import pandas as pd
import numpy as np
import regex as re
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import nltk
from nltk.corpus import stopwords
import contractions
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

def remover_caracteres_especiais(texto):
    return re.sub(r'[^a-z\s]', '', texto)

def remover_stopwords(texto, stop_words):
    palavras = nltk.word_tokenize(texto)
    palavras_filtradas = [palavra for palavra in palavras if palavra.lower() not in stop_words]
    return ' '.join(palavras_filtradas)

def remover_urls(texto):
    # Remove todos os urls que começam com 'www', 'https' e 'http'
    return re.sub(r'http\S+|www\S+|https\S+', '', texto, flags=re.MULTILINE)

def remover_contracoes(texto):
    return contractions.fix(texto)

def lematizar(texto):
    lemmatizer = WordNetLemmatizer()
    words = word_tokenize(texto)
    # Reduzir as palavras e retornar a lista de palavras reduzidas
    return ' '.join(lemmatizer.lemmatize(w) for w in words)


# Carrega os dados
sms = pd.read_csv('SMSSpamCollection', delimiter='\t', header=None, names=['Label', 'Mensagem'])

# Discretiza os labels
sms['Label'] = sms['Label'].map({'ham': 0, 'spam': 1})

# Cria uma nova coluna com texto em minúsculas
sms['Mensagem_Preprocessada'] = sms['Mensagem'].str.lower()

# Remove tudo que não é letra minúscula
sms['Mensagem_Preprocessada'] = sms['Mensagem_Preprocessada'].apply(remover_caracteres_especiais)

# Inicializa as stopwords
stop_words = set(stopwords.words('english'))

# Aplica as funções de pré-processamento
sms['Mensagem_Preprocessada'] = sms['Mensagem_Preprocessada'].apply(remover_stopwords, args=(stop_words,))
sms['Mensagem_Preprocessada'] = sms['Mensagem_Preprocessada'].apply(remover_contracoes)
sms['Mensagem_Preprocessada'] = sms['Mensagem_Preprocessada'].apply(lematizar)
sms['Mensagem_Preprocessada'] = sms['Mensagem_Preprocessada'].apply(remover_urls)

# Exibir as primeiras linhas do DataFrame resultante
print(sms.head())

   Label                                           Mensagem  \
0      0  Go until jurong point, crazy.. Available only ...   
1      0                      Ok lar... Joking wif u oni...   
2      1  Free entry in 2 a wkly comp to win FA Cup fina...   
3      0  U dun say so early hor... U c already then say...   
4      0  Nah I don't think he goes to usf, he lives aro...   

                              Mensagem_Preprocessada  
0  go jurong point crazy available bugis n great ...  
1                          ok lar joking wif you oni  
2  free entry wkly comp win fa cup final tkts st ...  
3            you dun say early hor you c already say  
4         nah do not think go usf life around though  


# Criar a tabela tf-idf sem vazar os dados

In [43]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

# Dividir em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(sms['Mensagem_Preprocessada'], sms['Label'], stratify=sms['Label'], test_size=0.2, random_state=42)

# Criar o vetor TF-IDF
vectorizer = TfidfVectorizer(stop_words='english')

# Ajustar e transformar os dados
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)


# Grid Search - diferentes modelos

In [48]:
#bibliotecas
from sklearn.metrics import f1_score, accuracy_score
from sklearn.model_selection import GridSearchCV 
from sklearn.pipeline import Pipeline

X = sms['Mensagem_Preprocessada']
y = sms['Label']

# KNN

In [50]:
from sklearn.neighbors import KNeighborsClassifier


# Criar um pipeline com TfidfVectorizer e KNeighborsClassifier
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words='english')),
    ('knn', KNeighborsClassifier())
])

# Definir os hiperparâmetros para o GridSearch
param_grid = {
    'knn__n_neighbors': [1, 3, 5, 7, 9, 11, 15],
    'knn__weights': ['uniform', 'distance'],
    'knn__metric': ['euclidean', 'manhattan']
}

# Configurar o GridSearchCV
grid_search = GridSearchCV(estimator=pipeline, param_grid=param_grid, 
                           scoring='f1_macro', cv=5, n_jobs=-1, verbose=0)

In [51]:

# Ajustar o GridSearch aos dados
grid_search.fit(X, y)

# Obter os melhores hiperparâmetros
print("Melhores Hiperparâmetros:", grid_search.best_params_)

# Fazer previsões com os melhores parâmetros
best_model = grid_search.best_estimator_

# Dividir em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Ajustar o modelo com os melhores parâmetros
best_model.fit(X_train, y_train)
y_pred = best_model.predict(X_test)

# Avaliar o modelo
acuracia = accuracy_score(y_test, y_pred)
f1_macro = f1_score(y_test, y_pred, average='macro')

print("Acurácia:", acuracia)
print("F1 Macro:", f1_macro)

Melhores Hiperparâmetros: {'knn__metric': 'euclidean', 'knn__n_neighbors': 1, 'knn__weights': 'uniform'}
Acurácia: 0.947085201793722
F1 Macro: 0.8666346354625976


# SVM

In [52]:
from sklearn import svm
from sklearn.svm import SVC  


# Criar um pipeline com TfidfVectorizer e SVC
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('svm', SVC())
])

# Definir os hiperparâmetros para o GridSearch
param_grid = {
    'svm__C': [0.1, 1, 10, 100],
    'svm__kernel': ['linear', 'rbf'],
    'svm__gamma': ['scale', 'auto']
}

# Configurar o GridSearchCV
grid_search = GridSearchCV(estimator=pipeline, param_grid=param_grid, 
                           scoring='f1_macro', cv=5, n_jobs=-1, verbose=0)

In [53]:
# Ajustar o GridSearch aos dados
grid_search.fit(X, y)

# Obter os melhores hiperparâmetros
print("Melhores Hiperparâmetros:", grid_search.best_params_)

# Fazer previsões com os melhores parâmetros
best_model = grid_search.best_estimator_

# Dividir em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Ajustar o modelo com os melhores parâmetros
best_model.fit(X_train, y_train)
y_pred = best_model.predict(X_test)

# Avaliar o modelo
acuracia = accuracy_score(y_test, y_pred)
f1_macro = f1_score(y_test, y_pred, average='macro')

print("Acurácia:", acuracia)
print("F1 Macro:", f1_macro)

Melhores Hiperparâmetros: {'svm__C': 1, 'svm__gamma': 'scale', 'svm__kernel': 'linear'}
Acurácia: 0.979372197309417
F1 Macro: 0.953174271320816


# Decision Tree

In [54]:
from sklearn.tree import DecisionTreeClassifier

# Criar um pipeline com TfidfVectorizer e DecisionTreeClassifier
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('dt', DecisionTreeClassifier())
])

# Definir os hiperparâmetros para o GridSearch
param_grid = {
    'dt__criterion': ['gini', 'entropy'],
    'dt__max_depth': [None, 10, 20, 30],
    'dt__min_samples_split': [2, 5, 10],
    'dt__min_samples_leaf': [1, 2, 4]
}

# Configurar o GridSearchCV
grid_search = GridSearchCV(estimator=pipeline, param_grid=param_grid, 
                           scoring='f1_macro', cv=5, n_jobs=-1, verbose=0)


In [55]:
# Ajustar o GridSearch aos dados
grid_search.fit(X, y)

# Obter os melhores hiperparâmetros
print("Melhores Hiperparâmetros:", grid_search.best_params_)

# Fazer previsões com os melhores parâmetros
best_model = grid_search.best_estimator_

# Dividir em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Ajustar o modelo com os melhores parâmetros
best_model.fit(X_train, y_train)
y_pred = best_model.predict(X_test)

# Avaliar o modelo
acuracia = accuracy_score(y_test, y_pred)
f1_macro = f1_score(y_test, y_pred, average='macro')

print("Acurácia:", acuracia)
print("F1 Macro:", f1_macro)

Melhores Hiperparâmetros: {'dt__criterion': 'gini', 'dt__max_depth': None, 'dt__min_samples_leaf': 1, 'dt__min_samples_split': 5}
Acurácia: 0.9497757847533632
F1 Macro: 0.8890152861713473


# Validação Cruzada

In [9]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import StratifiedKFold
from sklearn import svm
from sklearn.svm import SVC  
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score

# DataFrame com as colunas 'Mensagem' e 'Label'
X = sms['Mensagem_Preprocessada']
y = sms['Label']

# Hiperparâmetros definidos
best_params_svm = {'C': 1, 'gamma': 'scale', 'kernel': 'linear'}
best_params_knn = {'metric': 'euclidean', 'n_neighbors': 1, 'weights': 'uniform'}
best_params_tree = {'criterion': 'gini', 'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 5}

skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=123)

f1_score_svm = []
f1_score_knn = []
f1_score_tree = []

for train_index, test_index in skf.split(X, y):
    # Separação manual dos conjuntos de treino e teste    
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Vetorização desses conjuntos
    vectorizer = TfidfVectorizer(stop_words='english')
    
    # Ajustar e transformar os dados
    X_train_vect = vectorizer.fit_transform(X_train)
    X_test_vect = vectorizer.transform(X_test)

    # Modelo SVM
    model_svm = SVC(**best_params_svm)
    model_svm.fit(X_train_vect, y_train)
    y_pred_svm = model_svm.predict(X_test_vect)
    f1_score_svm.append(f1_score(y_test, y_pred_svm, average='macro'))

    # Modelo KNN
    model_knn = KNeighborsClassifier(**best_params_knn)
    model_knn.fit(X_train_vect, y_train)
    y_pred_knn = model_knn.predict(X_test_vect)
    f1_score_knn.append(f1_score(y_test, y_pred_knn, average='macro'))

    # Modelo Decision Tree
    model_tree = DecisionTreeClassifier(**best_params_tree)
    model_tree.fit(X_train_vect, y_train)
    y_pred_tree = model_tree.predict(X_test_vect)
    f1_score_tree.append(f1_score(y_test, y_pred_tree, average='macro'))

# Convertendo listas para arrays NumPy
f1_score_svm = np.array(f1_score_svm)
f1_score_knn = np.array(f1_score_knn)
f1_score_tree = np.array(f1_score_tree)

# Cálculo da média e do desvio padrão dos F1 scores
mean_f1_svm = np.mean(f1_score_svm)
std_f1_svm = np.std(f1_score_svm)

mean_f1_knn = np.mean(f1_score_knn)
std_f1_knn = np.std(f1_score_knn)

mean_f1_tree = np.mean(f1_score_tree)
std_f1_tree = np.std(f1_score_tree)

# Resultados
print(f"Média F1 Score SVM: {mean_f1_svm:.4f}, Desvio Padrão: {std_f1_svm:.4f}")
print(f"Média F1 Score KNN: {mean_f1_knn:.4f}, Desvio Padrão: {std_f1_knn:.4f}")
print(f"Média F1 Score Decision Tree: {mean_f1_tree:.4f}, Desvio Padrão: {std_f1_tree:.4f}")


Média F1 Score SVM: 0.9564, Desvio Padrão: 0.0105
Média F1 Score KNN: 0.8885, Desvio Padrão: 0.0161
Média F1 Score Decision Tree: 0.9278, Desvio Padrão: 0.0174


# Teste T

# codigo do Fonseca

In [11]:
import scipy.stats as stats
import itertools

# dataset = "sentistrength_twitter_2L"
# dataset = "vader_amazon_2L"
dataset = "luxury_beauty_2L"
# dataset = "digital_music_2L"

pastas = [
          'padraonovo',
          'rr_02',
          'e2sc_RL',
        #   'cnn',
          'nearmiss_1',
          'nearmiss_2',
         ]

saidaH = {}
for pasta in pastas:
    dirCls = f"resultados-undersampling-llm/out-gui-{pasta}/llama3.1/{dataset}"

    NUMFOLD = 10
    if dataset  in ["luxury_beauty_2L", "digital_music_2L"]:
        NUMFOLD = 5

    saidaH[pasta] = []
    for fold in range(NUMFOLD):
        with open(f"{dirCls}/out.fold={fold}.json", 'r') as arquivo:
            dados = json.load(arquivo)

            saidaH[pasta].append(dados["macro"])

saidaH_teste = []

pares = list(itertools.combinations(saidaH.keys(), 2))

# Número de testes realizados (correção de Bonferroni)
n_testes = len(pares)

for metodo_a, metodo_b in pares:
    valores_a = saidaH[metodo_a]
    valores_b = saidaH[metodo_b]
    
    # Teste t
    t_stat, p_val = stats.ttest_ind(valores_a, valores_b)
    
    # Correção de Bonferroni
    p_val_corrigido = p_val * n_testes
    
    if p_val_corrigido < 0.05:
        comparacao = "estatisticamente diferente"
    else:
        comparacao = "não estatisticamente diferente"
    
    saidaH_teste.append({
        'metodo_a': metodo_a,
        'metodo_b': metodo_b,
        'comparacao': comparacao
    })


print(f"{dataset}")
for resultado in saidaH_teste:
    if resultado['metodo_a'] != 'padraonovo': break
    print(f"Método A: {resultado['metodo_a']}, Método B: {resultado['metodo_b']}")
    print(f"Comparação: {resultado['comparacao']}")
    print("-" * 50)

FileNotFoundError: [Errno 2] No such file or directory: 'resultados-undersampling-llm/out-gui-padraonovo/llama3.1/luxury_beauty_2L/out.fold=0.json'

# Meu código

In [12]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVC  
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score
import scipy.stats as stats
import itertools

# DataFrame com as colunas 'Mensagem' e 'Label'
X = sms['Mensagem_Preprocessada']
y = sms['Label']

# Hiperparâmetros definidos
best_params_svm = {'C': 1, 'gamma': 'scale', 'kernel': 'linear'}
best_params_knn = {'metric': 'euclidean', 'n_neighbors': 1, 'weights': 'uniform'}
best_params_tree = {'criterion': 'gini', 'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 5}

skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=123)

# Dicionário para armazenar F1 scores
f1_scores = {
    'SVM': [],
    'KNN': [],
    'Decision Tree': []
}

for train_index, test_index in skf.split(X, y):
    # Separação manual dos conjuntos de treino e teste    
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Vetorização desses conjuntos
    vectorizer = TfidfVectorizer(stop_words='english')
    
    # Ajustar e transformar os dados
    X_train_vect = vectorizer.fit_transform(X_train)
    X_test_vect = vectorizer.transform(X_test)

    # Modelo SVM
    model_svm = SVC(**best_params_svm)
    model_svm.fit(X_train_vect, y_train)
    y_pred_svm = model_svm.predict(X_test_vect)
    f1_scores['SVM'].append(f1_score(y_test, y_pred_svm, average='macro'))

    # Modelo KNN
    model_knn = KNeighborsClassifier(**best_params_knn)
    model_knn.fit(X_train_vect, y_train)
    y_pred_knn = model_knn.predict(X_test_vect)
    f1_scores['KNN'].append(f1_score(y_test, y_pred_knn, average='macro'))

    # Modelo Decision Tree
    model_tree = DecisionTreeClassifier(**best_params_tree)
    model_tree.fit(X_train_vect, y_train)
    y_pred_tree = model_tree.predict(X_test_vect)
    f1_scores['Decision Tree'].append(f1_score(y_test, y_pred_tree, average='macro'))


# Convertendo listas para arrays NumPy
f1_score_svm = np.array(f1_scores['SVM'])
f1_score_knn = np.array(f1_scores['KNN'])
f1_score_tree = np.array(f1_scores['Decision Tree'])

# Cálculo da média e do desvio padrão dos F1 scores
mean_f1_svm = np.mean(f1_score_svm)
std_f1_svm = np.std(f1_score_svm)

mean_f1_knn = np.mean(f1_score_knn)
std_f1_knn = np.std(f1_score_knn)

mean_f1_tree = np.mean(f1_score_tree)
std_f1_tree = np.std(f1_score_tree)

# Resultados
print(f"Média F1 Score SVM: {mean_f1_svm:.4f}, Desvio Padrão: {std_f1_svm:.4f}")
print(f"Média F1 Score KNN: {mean_f1_knn:.4f}, Desvio Padrão: {std_f1_knn:.4f}")
print(f"Média F1 Score Decision Tree: {mean_f1_tree:.4f}, Desvio Padrão: {std_f1_tree:.4f}")

# Comparação estatística usando teste t
saidaH_teste = []
pares = list(itertools.combinations(['SVM', 'KNN', 'Decision Tree'], 2))
n_testes = len(pares)

for metodo_a, metodo_b in pares:
    valores_a = f1_scores[metodo_a]
    valores_b = f1_scores[metodo_b]
    
    # Teste t
    t_stat, p_val = stats.ttest_ind(valores_a, valores_b)
    
    # Correção de Bonferroni
    p_val_corrigido = p_val * n_testes
    
    if p_val_corrigido < 0.05:
        comparacao = "estatisticamente diferente"
    else:
        comparacao = "não estatisticamente diferente"
    
    saidaH_teste.append({
        'metodo_a': metodo_a,
        'metodo_b': metodo_b,
        'comparacao': comparacao,
        't_stat': t_stat,
        'p_val': p_val_corrigido
    })

# Exibir resultados do teste t
for resultado in saidaH_teste:
    print(f"Método A: {resultado['metodo_a']}, Método B: {resultado['metodo_b']}")
    print(f"Comparação: {resultado['comparacao']}, Estatística t: {resultado['t_stat']:.4f}, p-valor corrigido: {resultado['p_val']:.4f}")
    print("-" * 50)


Média F1 Score SVM: 0.9564, Desvio Padrão: 0.0105
Média F1 Score KNN: 0.8885, Desvio Padrão: 0.0161
Média F1 Score Decision Tree: 0.9192, Desvio Padrão: 0.0158
Método A: SVM, Método B: KNN
Comparação: estatisticamente diferente, Estatística t: 10.6058, p-valor corrigido: 0.0000
--------------------------------------------------
Método A: SVM, Método B: Decision Tree
Comparação: estatisticamente diferente, Estatística t: 5.8769, p-valor corrigido: 0.0000
--------------------------------------------------
Método A: KNN, Método B: Decision Tree
Comparação: estatisticamente diferente, Estatística t: -4.0832, p-valor corrigido: 0.0021
--------------------------------------------------


# Word Embeddings

In [33]:
#bibliotecas

from gensim.models import Word2Vec
import fasttext

# modelo Word2Vec

In [21]:
# Tokenizar as mensagens
sms['Tokens'] = sms['Mensagem_Preprocessada'].apply(lambda x: x.split())

# Treinar o modelo Word2Vec
modelo_w2v = Word2Vec(sentences=sms['Tokens'], vector_size=100, window=5, min_count=1, workers=4)

# Salvar o modelo se desejar
# modelo_w2v.save("modelo_w2v.model")


# Pulling

In [22]:
def get_sentence_vector(sentence, model):
    words = sentence.split()
    word_vectors = [model.wv[word] for word in words if word in model.wv]
    return np.mean(word_vectors, axis=0) if word_vectors else np.zeros(model.vector_size)

# Criar vetores de sentença
sms['Mensagem_Vector'] = sms['Mensagem_Preprocessada'].apply(lambda x: get_sentence_vector(x, modelo_w2v))


In [23]:
# Transformar em array
X = np.array(list(sms['Mensagem_Vector']))
y = sms['Label'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Hiperparâmetros definidos
best_params_svm = {'C': 1, 'gamma': 'scale', 'kernel': 'linear'}
best_params_knn = {'metric': 'euclidean', 'n_neighbors': 1, 'weights': 'uniform'}
best_params_tree = {'criterion': 'gini', 'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 5}


# KNN
knn_model = KNeighborsClassifier(**best_params_knn)
knn_model.fit(X_train, y_train)
y_pred_knn = knn_model.predict(X_test)

# SVM
svm_model = SVC(**best_params_svm)
svm_model.fit(X_train, y_train)
y_pred_svm = svm_model.predict(X_test)

# Decision Tree
tree_model = DecisionTreeClassifier(**best_params_tree)
tree_model.fit(X_train, y_train)
y_pred_tree = tree_model.predict(X_test)


# Avaliação do KNN
acuracia_knn = accuracy_score(y_test, y_pred_knn)
f1_macro_knn = f1_score(y_test, y_pred_knn, average='macro')
print("KNN - Acurácia:", acuracia_knn)
print("KNN - F1 Macro:", f1_macro_knn)

# Avaliação do SVM
acuracia_svm = accuracy_score(y_test, y_pred_svm)
f1_macro_svm = f1_score(y_test, y_pred_svm, average='macro')
print("SVM - Acurácia:", acuracia_svm)
print("SVM - F1 Macro:", f1_macro_svm)

# Avaliação da Decision Tree
acuracia_tree = accuracy_score(y_test, y_pred_tree)
f1_macro_tree = f1_score(y_test, y_pred_tree, average='macro')
print("Decision Tree - Acurácia:", acuracia_tree)
print("Decision Tree - F1 Macro:", f1_macro_tree)



KNN - Acurácia: 0.9291479820627803
KNN - F1 Macro: 0.856995816293564
SVM - Acurácia: 0.8663677130044843
SVM - F1 Macro: 0.46419990389235943
Decision Tree - Acurácia: 0.9327354260089686
Decision Tree - F1 Macro: 0.8591196939296382


# modelo FastTex

In [34]:
# Criar um arquivo temporário para o FastText
with open('temp_fasttext.txt', 'w') as f:
    for mensagem in sms['Mensagem_Preprocessada']:
        f.write(mensagem + '\n')

# Treinar o modelo FastText
modelo_fasttext = fasttext.train_unsupervised('temp_fasttext.txt', model='skipgram')

# Remover o arquivo temporário se não for mais necessário
import os
os.remove('temp_fasttext.txt')



Read 0M words
Number of words:  1568
Number of labels: 0
Progress: 100.0% words/sec/thread:   65216 lr:  0.000000 avg.loss:  2.840988 ETA:   0h 0m 0s


In [35]:
def get_sentence_vector_fasttext(sentence, model):
    words = sentence.split()
    word_vectors = [model.get_word_vector(word) for word in words if word in model.get_words()]
    return np.mean(word_vectors, axis=0) if word_vectors else np.zeros(model.get_dimension())

# Criar vetores de sentença
sms['Mensagem_Vector'] = sms['Mensagem_Preprocessada'].apply(lambda x: get_sentence_vector_fasttext(x, modelo_fasttext))

X = np.array(list(sms['Mensagem_Vector']))
y = sms['Label'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# KNN
knn_model = KNeighborsClassifier(**best_params_knn)
knn_model.fit(X_train, y_train)
y_pred_knn = knn_model.predict(X_test)

# SVM
svm_model = SVC(**best_params_svm)
svm_model.fit(X_train, y_train)
y_pred_svm = svm_model.predict(X_test)

# Decision Tree
tree_model = DecisionTreeClassifier(**best_params_tree)
tree_model.fit(X_train, y_train)
y_pred_tree = tree_model.predict(X_test)

# Avaliação do KNN
acuracia_knn = accuracy_score(y_test, y_pred_knn)
f1_macro_knn = f1_score(y_test, y_pred_knn, average='macro')
print("KNN - Acurácia:", acuracia_knn)
print("KNN - F1 Macro:", f1_macro_knn)

# Avaliação do SVM
acuracia_svm = accuracy_score(y_test, y_pred_svm)
f1_macro_svm = f1_score(y_test, y_pred_svm, average='macro')
print("SVM - Acurácia:", acuracia_svm)
print("SVM - F1 Macro:", f1_macro_svm)

# Avaliação da Decision Tree
acuracia_tree = accuracy_score(y_test, y_pred_tree)
f1_macro_tree = f1_score(y_test, y_pred_tree, average='macro')
print("Decision Tree - Acurácia:", acuracia_tree)
print("Decision Tree - F1 Macro:", f1_macro_tree)


KNN - Acurácia: 0.9515695067264573
KNN - F1 Macro: 0.8993843747493382
SVM - Acurácia: 0.9443946188340807
SVM - F1 Macro: 0.8632020959979736
Decision Tree - Acurácia: 0.9533632286995516
Decision Tree - F1 Macro: 0.8998618307426598
